<h5><b>Assignment 05:</b></h5>
 <h5><b>Event Extraction from Syntactic Dependencies</b></h5>
 <p>Shashikala Kankanamge (6648398)</p>

<b> Task 1: Getting the Model, First Parse</b>

In [3]:
# !pip install Spacy

In [4]:
import spacy

In [5]:
# !python -m spacy download es_core_news_md

In [6]:
model = spacy.load('es_core_news_md')

In [7]:
text_content = open("azuela1920_los-de-abajo.txt", "r", 
                    encoding="utf-8").read()
print(len(text_content))


201845


In [8]:
doc =model(text_content)
print(type(doc))
print(f"Total tokens: {len(doc)}")
print(f"Total sentences: {len(list(doc.sents))}")


<class 'spacy.tokens.doc.Doc'>
Total tokens: 45720
Total sentences: 1928


In [9]:
# Extract the sentence spans
  
sentences = [sent.text.strip() for sent in doc.sents]
sentences[:5]

['Los de Abajo\nMariano Azuela\n\nPRIMERA PARTE\n=============\n\nI\n—Te digo que no es un animal... Oye cómo ladra el Palomo... Debe ser algún cristiano...\nLa mujer fijaba sus pupilas en la oscuridad de la sierra.',
 '— ¿Y que fueran siendo federales?',
 '—repuso un hombre que, en cuclillas, yantaba en un rincón, una\ncazuela en la diestra y tres tortillas en taco en la otra mano.',
 'La mujer no le contestó; sus sentidos estaban puestos fuera de la casuca.',
 'Se oyó un ruido de pesuñas en el pedregal cercano, y el Palomo ladró con más rabia.']

In [10]:
output_file_with_newLine = "sentences_with_newLine.txt"
with open(output_file_with_newLine, "w",
          encoding="utf-8") as output_file:
    for sentence in sentences:
        output_file.write(sentence + "\n")

print(f"Extract and save file to :  {output_file_with_newLine}")

Extract and save file to :  sentences_with_newLine.txt


<b>Task 2: Understanding Sentence Segmentation</b>

In [12]:
output_file_with_space = "sentences_with_space.txt"
with open(output_file_with_space, "w", 
          encoding="utf-8") as output_file:
    for sentence in sentences:
        # print(sentence)
        output_file.write(sentence + " ")

print(f"Extract and save file to :  {output_file_with_space}")


Extract and save file to :  sentences_with_space.txt


In [13]:
with open("./sentences_with_newLine.txt", "r", 
          encoding="utf-8") as file1:
    sentence_newLine = file1.readlines()

with open("./sentences_with_space.txt", "r",
          encoding="utf-8") as file2:
    sentence_with_space = file2.readlines()

print(f"Original Sentence Count (with newlines): {len(sentences)}")
print(f"Original Sentence Count (with newlines): {len(sentence_newLine)}")
print(f"Sentence Count (with spaces): {len(sentence_with_space)}")


Original Sentence Count (with newlines): 1928
Original Sentence Count (with newlines): 3951
Sentence Count (with spaces): 2024


<b>Task 3: Extracting Verb-Subject Pairs</b>

In [15]:
verb_subject_pairs = []
for token in doc:
    if token.pos_ == "VERB" and any(child.dep_ == "nsubj" for 
                                    child in token.children):
        subject = [child for child in token.children if 
                   child.dep_ == "nsubj"][0]
        verb_subject_pairs.append((token.lemma_, subject.lemma_))

print('Total pair length :',len(verb_subject_pairs))
verb_subject_pairs[:10]



Total pair length : 2135


[('ladrar', 'Palomo'),
 ('fijar', 'mujer'),
 ('reponer', 'hombre'),
 ('yantar', 'que'),
 ('contestar', 'mujer'),
 ('oir', 'ruido'),
 ('acabar', 'hombre'),
 ('acercar', 'cántaro'),
 ('pronunciar', 'él'),
 ('alumbrar', 'cuartito')]

<b>Task 4: From Actions to Characters</b>

In [17]:
from collections import Counter
verb_counter = Counter(verb for verb, subject in 
                       verb_subject_pairs)
most_common_verbs = verb_counter.most_common(10)
most_common_verbs

[('decir', 126),
 ('tener', 53),
 ('hacer', 50),
 ('dar', 45),
 ('saber', 44),
 ('querer', 38),
 ('venir', 36),
 ('ir', 34),
 ('dejar', 28),
 ('volver', 28)]

In [18]:

verbs_of_interest = ["gritar", "preguntar", "responder"]
subjects_by_verb = {verb: [] for verb in verbs_of_interest}

for verb, subject in verb_subject_pairs:
    if verb in verbs_of_interest:
        subjects_by_verb[verb].append(subject)

for verb, subjects in subjects_by_verb.items():
    subject_counts = Counter(subjects)
    print(f"\nMost Common Subjects for '{verb}':")
    for subject, count in subject_counts.most_common(3):  # Top 3
        print(f"{subject}: {count}")


Most Common Subjects for 'gritar':
Manteca: 2
Demetrio: 2
María: 2

Most Common Subjects for 'preguntar':
Pancracio: 4
Anastasio: 3
Demetrio: 3

Most Common Subjects for 'responder':
Demetrio: 3
Pancracio: 3
Carranzo: 1


<li>Who shouts the Most: Manteca, Demetrio, and María.</li>
<li>Who asks the Most Questions: Pancracio</li>
<li>Who answers the Most: Demetrio & Pancracio </li>

<b>Task 5: Foreground and Background Events</b>

In [21]:
foreground_pairs = []
background_pairs = []

for verb, subject in verb_subject_pairs:
    verb_token = next((token for token in doc if token.lemma_ == verb), None)
    
    subject_token = next((token for token in doc if token.lemma_ == subject), None)
    
    if verb_token and verb_token.pos_ == "VERB" and "Tense=Past" in verb_token.morph:
        foreground_pairs.append((verb, subject))
    
    if verb_token and verb_token.pos_ == "VERB" and "Tense=Imp" in verb_token.morph:
        background_pairs.append((verb, subject))





In [22]:
foreground_pairs[:5]

[('reponer', 'hombre'),
 ('contestar', 'mujer'),
 ('oir', 'ruido'),
 ('acabar', 'hombre'),
 ('acercar', 'cántaro')]

In [23]:
background_pairs[:5]

[('fijar', 'mujer'),
 ('yantar', 'que'),
 ('alumbrar', 'cuartito'),
 ('servir', 'que'),
 ('dormir', 'hilacha')]

In [24]:



from collections import Counter

foreground_verbs = Counter(verb for verb, subject in foreground_pairs)
background_verbs = Counter(verb for verb, subject in background_pairs)



print("background_verbs:", len(foreground_verbs))




background_verbs: 246


In [25]:
print("foreground_verbs:", len(foreground_verbs))


foreground_verbs: 246


In [26]:

exclusive_foreground_verbs = {
    verb: count
    for verb, count in foreground_verbs.items()
    if count > 3 and verb not in background_verbs
}

exclusive_background_verbs = {
    verb: count
    for verb, count in background_verbs.items()
    if count > 3 and verb not in foreground_verbs
}





In [27]:
print("Exclusive Foreground Verbs:")
exclusive_foreground_verbs

Exclusive Foreground Verbs:


{'reponer': 11,
 'contestar': 16,
 'oir': 15,
 'acabar': 19,
 'acercar': 7,
 'pronunciar': 10,
 'saltar': 4,
 'lanzar': 7,
 'llegar': 15,
 'quedar': 26,
 'matar': 6,
 'entrar': 13,
 'preguntar': 22,
 'llenar': 7,
 'exclamar': 21,
 'poner': 18,
 'creer': 4,
 'tirar': 6,
 'comenzar': 22,
 'despertar': 10,
 'ascender': 4,
 'detener': 5,
 'salir': 19,
 'quemar': 5,
 'observar': 9,
 'gritar': 21,
 'tender': 5,
 'aparecer': 7,
 'asomar': 6,
 'ordenar': 5,
 'caer': 17,
 'volver': 28,
 'regresar': 4,
 'tomar': 10,
 'coger': 6,
 'alzar': 9,
 'abrir': 10,
 'responder': 18,
 'levantar': 15,
 'interrumpir': 4,
 'inquirir': 6,
 'sonreír': 7,
 'encontrar': 9,
 'sacar': 10,
 'sentir': 8,
 'reír': 4,
 'suceder': 5,
 'proseguir': 6,
 'brillar': 6,
 'clavar': 4,
 'escapar': 4}

In [28]:
print("Exclusive Background Verbs:")
exclusive_background_verbs

Exclusive Background Verbs:


{'servir': 5,
 'dormir': 8,
 'vestir': 5,
 'llevar': 12,
 'estar': 16,
 'seguir': 20,
 'permanecer': 4,
 'clamar': 12,
 'tocar': 6,
 'pedir': 8,
 'meter': 8,
 'ocurrir': 4,
 'cubrir': 6,
 'escuchar': 4,
 'remover': 6,
 'pensar': 5,
 'reir': 4,
 'maté': 4}

<b> Task 6: Extraction of Key Events</b>

In [30]:
num_chunks = 10
chunk_size = len(sentences) // num_chunks
remainder = len(sentences) % num_chunks

print(chunk_size)

192


In [31]:

chunks = []
start = 0
for i in range(num_chunks):
    end = start + chunk_size + (1 if i < remainder else 0)
    chunks.append(sentences[start:end])
    start = end

for i, chunk in enumerate(chunks):
    print(f"Chunk {i + 1} has {len(chunk)} sentences.")

Chunk 1 has 193 sentences.
Chunk 2 has 193 sentences.
Chunk 3 has 193 sentences.
Chunk 4 has 193 sentences.
Chunk 5 has 193 sentences.
Chunk 6 has 193 sentences.
Chunk 7 has 193 sentences.
Chunk 8 has 193 sentences.
Chunk 9 has 192 sentences.
Chunk 10 has 192 sentences.


In [32]:
verbs_to_include = {verb for verb, count in
                    verb_counter.items() if count >= 5}

verbs_to_include

{'abrir',
 'acabar',
 'acercar',
 'alzar',
 'andar',
 'aparecer',
 'apurar',
 'asomar',
 'bajar',
 'brillar',
 'buscar',
 'caer',
 'cantar',
 'clamar',
 'coger',
 'comenzar',
 'conocer',
 'contar',
 'contestar',
 'correr',
 'cuadrar',
 'cubrir',
 'dar',
 'decir',
 'dejar',
 'desaparecer',
 'despertar',
 'detener',
 'dormir',
 'echar',
 'encontrar',
 'entender',
 'entrar',
 'esperar',
 'estar',
 'exclamar',
 'faltar',
 'gritar',
 'hablar',
 'hacer',
 'hundir',
 'inquirir',
 'ir',
 'lanzar',
 'levantar',
 'llamar',
 'llegar',
 'llenar',
 'llevar',
 'matar',
 'meter',
 'observar',
 'ofrecer',
 'oir',
 'ordenar',
 'parecer',
 'pasar',
 'pedir',
 'pensar',
 'perder',
 'poner',
 'preguntar',
 'pronunciar',
 'proseguir',
 'quedar',
 'quemar',
 'querer',
 'remover',
 'reponer',
 'responder',
 'robar',
 'saber',
 'sacar',
 'salir',
 'seguir',
 'sentir',
 'servir',
 'sonreír',
 'suceder',
 'tender',
 'tener',
 'tirar',
 'tocar',
 'tomar',
 'traer',
 'venir',
 'ver',
 'vestir',
 'vivir',
 'volver

In [44]:
import math
prominent_verbs_per_chunk = []

for chunk in chunks:
    chunk_verbs = []  
    chunk_doc = model(" ".join(chunk)) 
    
    for sent in chunk_doc.sents:
        for token in sent:
            if token.pos_ == "VERB":
                chunk_verbs.append(token.lemma_)
                

    chunk_verb_counts = Counter(chunk_verbs)

    log_ratios = {}
    for verb, chunk_count in chunk_verb_counts.items():
        if verb in verbs_to_include:
            overall_count = verb_counter.get(verb, 0)  
            log_ratio = math.log(chunk_count / overall_count) 
            log_ratios[verb] = log_ratio
    
    top_verbs = sorted(log_ratios.items(),  reverse=True)[:5]
    prominent_verbs_per_chunk.append(top_verbs)

for i, top_verbs in enumerate(prominent_verbs_per_chunk):
    print(f"Chunk {i+1}:")
    for verb, log_ratio in top_verbs:
        print(f"  {verb}: log:  {log_ratio:.2f}")


Chunk 1:
  volver: log:  -2.23
  vivir: log:  -0.92
  vestir: log:  -0.51
  ver: log:  -1.19
  venir: log:  -1.50
Chunk 2:
  volver: log:  -1.54
  vivir: log:  -0.51
  ver: log:  -0.94
  venir: log:  -1.64
  tomar: log:  -0.69
Chunk 3:
  volver: log:  -1.95
  vivir: log:  -1.61
  vestir: log:  -1.61
  ver: log:  -0.50
  venir: log:  -1.39
Chunk 4:
  volver: log:  -3.33
  vivir: log:  -1.61
  ver: log:  -0.94
  venir: log:  -2.20
  traer: log:  -0.92
Chunk 5:
  volver: log:  -1.54
  vivir: log:  -1.61
  ver: log:  -0.74
  venir: log:  -1.28
  traer: log:  -1.20
Chunk 6:
  volver: log:  -1.39
  vivir: log:  -1.61
  ver: log:  -1.06
  venir: log:  -1.79
  traer: log:  -1.61
Chunk 7:
  volver: log:  -1.72
  vestir: log:  -0.92
  ver: log:  -0.83
  venir: log:  -1.64
  traer: log:  -1.20
Chunk 8:
  volver: log:  -1.95
  vivir: log:  -1.61
  vestir: log:  -0.92
  ver: log:  -0.94
  venir: log:  -1.97
Chunk 9:
  volver: log:  -1.72
  ver: log:  -1.06
  venir: log:  -2.48
  traer: log:  -0.92
